# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint
import pathlib
from tqdm.notebook import tqdm

output_data_file = "output_data/cities.csv"

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = pathlib.Path(output_data_file)
original_df = pd.read_csv(filepath)
original_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hechingen,48.35,8.96,287.59,17,66,1.18,DE,1588986961
1,sisimiut,66.94,-53.67,275.92,73,100,1.06,GL,1588986961
2,sterling,39.01,-77.43,282.04,87,90,4.10,US,1588986959
3,shirokiy,49.76,129.52,289.26,45,100,3.37,RU,1588986962
4,stafford,52.81,-2.12,284.26,89,0,0.89,GB,1588986735


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
weather_df = original_df.copy()
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[['Lat', 'Lng']].astype(float)
locations.dropna()
humidity_num = weather_df['Humidity'].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(34, -118), zoom_level=1.2)

# humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_num)
humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_num, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
max_temp = (80 + 459.67)*5/9
min_temp = (70 + 459.67)*5/9
max_wind_speed = 10
cloudiness = 0

temp_boolean = ((weather_df['Max Temp'] >= min_temp) & (weather_df['Max Temp'] < max_temp))
wind_speed_boolean = (weather_df['Wind Speed'] <= max_wind_speed)
cloudiness_boolean = (weather_df['Cloudiness'] == cloudiness)

weather_filtered_df = weather_df.loc[(temp_boolean & wind_speed_boolean & cloudiness_boolean),:]
weather_filtered_df = weather_filtered_df.reset_index(drop=True)
weather_filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,baghdad,33.34,44.40,299.15,39,0,2.60,IQ,1588986973
1,dehloran,32.69,47.27,298.31,27,0,1.63,IR,1588987044
2,flinders,-34.58,150.86,297.59,50,0,4.92,AU,1588987060
3,camalu,30.85,-116.07,295.37,48,0,1.79,MX,1588987061
4,tikrit,34.61,43.68,297.30,36,0,3.52,IQ,1588987074
5,poum,-20.23,164.02,297.10,66,0,0.81,NC,1588987081
6,namibe,-15.20,12.15,299.10,55,0,1.20,AO,1588987167
7,badarpur,24.90,92.60,299.13,52,0,1.24,IN,1588987173
8,emerald,-23.53,148.17,298.15,64,0,5.10,AU,1588987196
9,guerrero negro,27.98,-114.06,295.09,62,0,7.60,MX,1588987052


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = pd.DataFrame()
target_search = 'Hotel'

lat = weather_filtered_df['Lat']
lng = weather_filtered_df['Lng']

for i in tqdm(range(len(weather_filtered_df['City']))):
    # redefine params
    params = {
        "location": f'{lat[i]},{lng[i]}',
        "keyword": target_search,
        "radius": 50000,
        "key": g_key
    }
    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params).json()
    hotel_df = hotel_df.append([[
        hotel['results'][0]['name'],
        hotel['results'][0]['geometry']['location']['lat'],
        hotel['results'][0]['geometry']['location']['lng'],
        weather_filtered_df['City'][i],
        weather_filtered_df['Country'][i]
    ]])

In [13]:
hotel_df.columns=['Hotel Name', 'Lat', 'Lng', 'City', 'Country']
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Babylon Rotana,33.291224,44.390489,baghdad,IQ
1,هتل سعید,32.685047,47.273764,dehloran,IR
2,Headlands Hotel Austinmer Beach,-34.301443,150.940377,flinders,AU
3,Mision Santa Maria,30.400381,-115.911920,camalu,MX
4,فندق تكريت بلازا,34.605165,43.671440,tikrit,IQ
5,Hotel Karem Bay,-20.547754,164.274260,poum,NC
6,Hotel ibis Styles Iu Namibe,-15.190917,12.151250,namibe,AO
7,THE CACHAR CLUB,24.829513,92.800061,badarpur,IN
8,Emerald Star Hotel,-23.526018,148.159839,emerald,AU
9,Hotel Los Caracoles,27.967758,-114.028336,guerrero negro,MX


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Create a combined map
fig = gmaps.figure(center=(34, -118), zoom_level=1.2)

fig.add_layer(humidity_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))